In [1]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
from utils import TextPairDataset, collate_fn_for_text_pair
from utils import DictMaker
from models import TransformerModel

In [3]:
filename = "C://NLP//clean_chat_corpus//chatterbot.tsv"
dt = TextPairDataset(filename)

sentence to index: 100%|██████████| 564/564 [00:00<00:00, 141372.58it/s]


In [4]:
data_iter = DataLoader(dt, batch_size=128, collate_fn=collate_fn_for_text_pair, shuffle=True)

In [5]:
for batch in data_iter:
    print(batch[0])
    print(batch[0].shape)
    break

tensor([[  1,   1,   1,  ...,   1,   1,   1],
        [663,  30,   3,  ..., 185, 185, 148],
        [179, 130,   4,  ..., 108, 108,   2],
        ...,
        [ 17,   0,   0,  ...,   0,   0,   0],
        [389,   0,   0,  ...,   0,   0,   0],
        [  2,   0,   0,  ...,   0,   0,   0]])
torch.Size([22, 128])


In [6]:
mask = batch[0] == 0
print(mask)

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False,  True,  True,  ...,  True,  True,  True],
        [False,  True,  True,  ...,  True,  True,  True],
        [False,  True,  True,  ...,  True,  True,  True]])


In [7]:
print(dt.voc.num_words)

1318


In [8]:
emb = nn.Embedding(1318, 10)

In [9]:
a = emb(batch[0])
print(a.shape)

torch.Size([22, 128, 10])


In [10]:

model = TransformerModel(dt.voc.num_words, ninp=16, nhead=4, nhid=256, nlayers=6)

In [12]:
print(batch[0].size())
print(batch[1].size())
output = model(batch[0], batch[1])

torch.Size([22, 128])
torch.Size([69, 128])
output torch.Size([69, 128, 16])


In [12]:
print(output.size())

torch.Size([69, 128, 1318])


In [13]:
output_dim = output.shape[-1]
output2 = output.contiguous().view(-1, output_dim)
print(output2.size())

torch.Size([8832, 1318])


In [15]:
print(batch[1].contiguous().view(-1).size())

torch.Size([8832])


In [16]:
loss = F.cross_entropy(output2, batch[1].contiguous().view(-1))

In [17]:
print(loss)

tensor(6.7771, grad_fn=<NllLossBackward>)


In [13]:
pred = output.argmax(2)
print(pred.size())

torch.Size([69, 128])


In [14]:
pred = pred[:, -1]
print(pred.size())

torch.Size([69])
